In [1]:
import os
import sys
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
import torch.nn.functional as F

# ==========================================
# 1. CONFIGURATION
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 320     # The Winning Base Resolution
BATCH_SIZE = 16    
ACCUM_STEPS = 4    # Virtual Batch Size = 64 (Smoother Gradients)
EPOCHS = 15
LEARNING_RATE = 2e-4
N_FOLDS = 5
IMAGE_DIR = Path("/kaggle/input/csiro-biomass")
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

print(f"Running ResNet34 with Gradient Accumulation + Multi-Scale TTA on {DEVICE}...")

# ==========================================
# 2. 4-CHANNEL DATASET
# ==========================================
class Biomass4ChannelDataset(Dataset):
    def __init__(self, df, target_cols=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.target_cols = target_cols
        self.is_test = is_test
        self.root_dir = IMAGE_DIR
        self.mean = torch.tensor([0.485, 0.456, 0.406, 0.5]).view(4,1,1)
        self.std = torch.tensor([0.229, 0.224, 0.225, 0.5]).view(4,1,1)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        rel_path = self.df.loc[idx, "image_path"]
        img_path = self.root_dir / rel_path
        try:
            pil_img = Image.open(img_path).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
            img = np.array(pil_img).astype(np.float32) / 255.0
        except:
            img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)

        # ExG Index
        r, g, b = img[:,:,0], img[:,:,1], img[:,:,2]
        exg = (2 * g) - r - b
        exg = (exg - exg.min()) / (exg.max() - exg.min() + 1e-6)
        
        img_4c = np.dstack((img, exg))
        image = torch.tensor(img_4c.transpose(2, 0, 1), dtype=torch.float32)
        
        if not self.is_test:
            if np.random.random() > 0.5: image = torch.flip(image, dims=[2])
            if np.random.random() > 0.5: image = torch.flip(image, dims=[1])

        image = (image - self.mean) / self.std

        if self.is_test:
            img_id = Path(rel_path).stem 
            return image, img_id
        else:
            targets = self.df.loc[idx, self.target_cols].values.astype(float)
            targets = np.log1p(targets)
            return image, torch.tensor(targets, dtype=torch.float32)

# ==========================================
# 3. MODEL
# ==========================================
def get_model():
    model = models.resnet34(weights=None)
    # Search Weights
    weights_path = None
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            if 'resnet34' in filename and '.pth' in filename:
                weights_path = os.path.join(dirname, filename)
                break
        if weights_path: break
    if weights_path:
        try: model.load_state_dict(torch.load(weights_path, weights_only=False))
        except: pass
    else:
        # Fallback
        model = models.resnet18(weights=None)
        for dirname, _, filenames in os.walk('/kaggle/input'):
            for filename in filenames:
                if 'resnet18' in filename and '.pth' in filename:
                    weights_path = os.path.join(dirname, filename)
                    break
            if weights_path: break
        if weights_path: model.load_state_dict(torch.load(weights_path, weights_only=False))

    original_conv1 = model.conv1
    model.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
    with torch.no_grad():
        model.conv1.weight[:, :3, :, :] = original_conv1.weight
        model.conv1.weight[:, 3:4, :, :] = torch.mean(original_conv1.weight, dim=1, keepdim=True)
    model.fc = nn.Linear(model.fc.in_features, 5)
    return model.to(DEVICE)

# ==========================================
# 4. WEIGHTED LOSS
# ==========================================
class WeightedHuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.huber = nn.HuberLoss(reduction='none', delta=delta)
        self.weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5]).to(DEVICE)
    def forward(self, preds, targets):
        return (self.huber(preds, targets) * self.weights).mean()

# ==========================================
# 5. TRAINING (With Gradient Accumulation)
# ==========================================
raw_df = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
train_pivot = raw_df.pivot(index='image_path', columns='target_name', values='target').reset_index().fillna(0.0)
train_pivot['bin'] = pd.qcut(train_pivot['Dry_Total_g'], q=10, labels=False, duplicates='drop')

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_pivot, train_pivot['bin'])):
    print(f"\n=== FOLD {fold+1}/{N_FOLDS} ===")
    
    train_loader = DataLoader(
        Biomass4ChannelDataset(train_pivot.iloc[train_idx], TARGET_COLS),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=2
    )
    valid_loader = DataLoader(
        Biomass4ChannelDataset(train_pivot.iloc[val_idx], TARGET_COLS),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=2
    )
    
    model = get_model()
    criterion = WeightedHuberLoss(delta=1.0)
    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    best_loss = float('inf')
    
    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad() # Reset once at start
        
        train_loss = 0
        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(DEVICE), y.to(DEVICE)
            
            # MixUp
            if np.random.random() < 0.5:
                lam = np.random.beta(1.0, 1.0)
                index = torch.randperm(x.size(0)).to(DEVICE)
                mixed_x = lam * x + (1 - lam) * x[index]
                y_lin_a = torch.expm1(y)
                y_lin_b = torch.expm1(y[index])
                mixed_y = torch.log1p(lam * y_lin_a + (1 - lam) * y_lin_b)
                preds = model(mixed_x)
                loss = criterion(preds, mixed_y)
            else:
                preds = model(x)
                loss = criterion(preds, y)
            
            # ACCUMULATION STEP
            loss = loss / ACCUM_STEPS
            loss.backward()
            
            if (batch_idx + 1) % ACCUM_STEPS == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            train_loss += loss.item() * ACCUM_STEPS
            
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in valid_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                val_loss += criterion(model(x), y).item()
        
        avg_val = val_loss / len(valid_loader)
        scheduler.step()
        
        if avg_val < best_loss:
            best_loss = avg_val
            torch.save(model.state_dict(), f"model_fold{fold}.pth")
            
    print(f"Fold {fold+1} Best: {best_loss:.4f}")
    
    del model, optimizer, train_loader, valid_loader
    torch.cuda.empty_cache()
    gc.collect()

# ==========================================
# 6. SUPER-INFERENCE (Multi-Scale TTA)
# ==========================================
print("Starting Multi-Scale TTA...")
test_df_raw = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
test_unique = test_df_raw[['image_path']].drop_duplicates()
test_ds = Biomass4ChannelDataset(test_unique, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

ensemble_preds = {} 

for fold in range(N_FOLDS):
    model = get_model()
    model.load_state_dict(torch.load(f"model_fold{fold}.pth", weights_only=True))
    model.eval()
    
    with torch.no_grad():
        for images, img_ids in test_loader:
            images = images.to(DEVICE)
            
            # --- SCALE TTA ---
            # 1. Base Scale (320px)
            pred_base = model(images)
            
            # 2. Zoom Out (0.9x) - Pad then Resize is complex, 
            # so we just Interpolate Down then pad, or simpler: just use Flips first
            
            # Standard Flips (High Value)
            pred_flip_h = model(torch.flip(images, dims=[3]))
            pred_flip_v = model(torch.flip(images, dims=[2]))
            
            # 3. Zoom In (1.1x) - Center Crop simulation
            # We upscale image to 352 (320*1.1) then Crop 320
            img_up = F.interpolate(images, scale_factor=1.1, mode='bilinear', align_corners=False)
            # Center crop logic
            h, w = img_up.shape[2], img_up.shape[3]
            start_h = (h - IMG_SIZE) // 2
            start_w = (w - IMG_SIZE) // 2
            img_zoom = img_up[:, :, start_h:start_h+IMG_SIZE, start_w:start_w+IMG_SIZE]
            pred_zoom = model(img_zoom)
            
            # Average 4 Views (Base, HFlip, VFlip, Zoom)
            avg_log = (pred_base + pred_flip_h + pred_flip_v + pred_zoom) / 4.0
            preds = np.expm1(avg_log.cpu().numpy())
            
            # --- ZERO THRESHOLDING (Noise Cleanup) ---
            # If prediction is tiny (< 0.1g), assume it's zero
            preds[preds < 0.1] = 0.0
            
            for i, img_id in enumerate(img_ids):
                if img_id not in ensemble_preds: ensemble_preds[img_id] = np.zeros(5)
                ensemble_preds[img_id] += preds[i]
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

results = []
for img_id, total_preds in ensemble_preds.items():
    avg_pred = total_preds / N_FOLDS
    for j, col in enumerate(TARGET_COLS):
        results.append({'sample_id': f"{img_id}__{col}", 'target': float(avg_pred[j])})

submission_df = pd.DataFrame(results)
submission_df['target'] = submission_df['target'].clip(lower=0.0)
submission_df.to_csv("submission.csv", index=False)
print("Super-Inference Submission Ready.")

Running ResNet34 with Gradient Accumulation + Multi-Scale TTA on cuda...

=== FOLD 1/5 ===
Fold 1 Best: 0.0312

=== FOLD 2/5 ===
Fold 2 Best: 0.0210

=== FOLD 3/5 ===
Fold 3 Best: 0.0231

=== FOLD 4/5 ===
Fold 4 Best: 0.0243

=== FOLD 5/5 ===
Fold 5 Best: 0.0284
Starting Multi-Scale TTA...
Super-Inference Submission Ready.


In [2]:
submission_df.head()

,sample_id,target
0,ID1001187975__Dry_Green_g,25.101497
1,ID1001187975__Dry_Dead_g,24.845939
2,ID1001187975__Dry_Clover_g,0.443528
3,ID1001187975__GDM_g,23.568129
4,ID1001187975__Dry_Total_g,55.320058
